In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
project_dir = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..", "..")))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_PATH,
)
from experiments.utils.parser import Parser

In [2]:
series = 71
experiment_id = 1
config_key_mapper = "key_config_mapper.csv"
model_name = "yolo"
series_path = os.path.join(NODE_PROFILING_RESULTS_PATH, "series", str(series))
loader = Parser(
    series_path=series_path, config_key_mapper=config_key_mapper, model_name=model_name
)
results = loader.result_processing()
key_config_df = loader.key_config_mapper()
# print(results.columns)
# print(key_config_df.columns)
# results

In [3]:
results.columns

Index(['experiment_id', 'client_to_model_avg', 'client_to_model_p99',
       'client_to_model_p95', 'client_to_model_p50', 'client_to_model_var',
       'client_to_model_max', 'client_to_model_min', 'model_latencies_avg',
       'model_latencies_p99', 'model_latencies_p95', 'model_latencies_p50',
       'model_latencies_var', 'model_latencies_max', 'model_latencies_min',
       'model_to_client_avg', 'model_to_client_p99', 'model_to_client_p95',
       'model_to_client_p50', 'model_to_client_var', 'model_to_client_max',
       'model_to_client_min', 'e2e_latencies_avg', 'e2e_latencies_p99',
       'e2e_latencies_p95', 'e2e_latencies_p50', 'e2e_latencies_var',
       'e2e_latencies_max', 'e2e_latencies_min', 'start_time', 'end_time',
       'duration', 'timeout_count', 'cpu_usage_count_avg',
       'cpu_usage_count_p99', 'cpu_usage_count_p95', 'cpu_usage_count_p50',
       'cpu_usage_count_var', 'cpu_usage_count_max', 'cpu_usage_count_min',
       'cpu_usage_rate_avg', 'cpu_usage_rate_p

In [4]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config)

File name: 1.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1', '2', '4', '8'],
    'data_type': 'image',
    'max_batch_size': ['1', '2', '8', '32', '64'],
    'max_batch_time': ['1'],
    'memory_request': ['10Gi'],
    'mode': 'exponential',
    'model_variants': ['yolov5n', 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x'],
    'node_name': 'yolo',
    'num_interop_threads': ['1'],
    'num_threads': ['1'],
    'pipeline_name': 'video',
    'repetition': 1,
    'replicas': [1],
    'series': 71,
    'series_meta': 'checking the effect of changing all variables under '
                   'arrival rate 1,\\n cpu type: Intel(R) Xeon(R) Gold 6126 '
                   'CPU @ 2.60GHz',
    'timeout': 1,
    'use_threading': 'True',
    'workload_config': {'load_duration': 60, 'loads_to_test': [1, 10, 20]},
    'workload_type': 'static'}
File name: 0.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1', '2', '4', '8'],
    'data_type': 'image',
    'max_batch_size': ['1', '2', '8',

In [5]:
display(key_config_df)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas,no_engine,mode,data_type,benchmark_duration
0,1,video,yolo,yolov5n,1,10Gi,1,1,1,60,71,checking the effect of changing all variables ...,1,False,exponential,image,1
1,2,video,yolo,yolov5n,1,10Gi,1,1,10,60,71,checking the effect of changing all variables ...,1,False,exponential,image,1
2,3,video,yolo,yolov5n,1,10Gi,1,1,20,60,71,checking the effect of changing all variables ...,1,False,exponential,image,1
3,4,video,yolo,yolov5n,2,10Gi,1,1,1,60,71,checking the effect of changing all variables ...,1,False,exponential,image,1
4,5,video,yolo,yolov5n,2,10Gi,1,1,10,60,71,checking the effect of changing all variables ...,1,False,exponential,image,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,video,yolo,yolov5x,4,10Gi,64,1,10,60,71,checking the effect of changing all variables ...,1,False,exponential,image,1
296,297,video,yolo,yolov5x,4,10Gi,64,1,20,60,71,checking the effect of changing all variables ...,1,False,exponential,image,1
297,298,video,yolo,yolov5x,8,10Gi,64,1,1,60,71,checking the effect of changing all variables ...,1,False,exponential,image,1
298,299,video,yolo,yolov5x,8,10Gi,64,1,10,60,71,checking the effect of changing all variables ...,1,False,exponential,image,1


In [6]:
experiment_ids = key_config_df[
    (key_config_df["model_variant"] == "yolov5m")
    & (key_config_df["cpu_request"] == 1)
    & (key_config_df["load"] == 20)
    & (key_config_df["max_batch_size"] == 1)
]["experiment_id"].tolist()
metadata_columns = ["model_variant", "max_batch_size", "cpu_request", "load"]
results_columns = [
    # "cpu_usage_count_avg",
    "model_latencies_avg",
    "model_latencies_p99",
    "throughput_max",
]
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns,
)
output["expected_max_throughput"] = (
    1 / output["model_latencies_p99"] * output["max_batch_size"]
)
# output.sort_values(by='throughput')
display(output)
# ax = output.plot.bar(x='max_batch_size', y=['throughput'])
# ax.set_xlabel("Max Batch Size")
# ax.set_ylabel("Throughput (RPS)")

,model_variant,max_batch_size,cpu_request,load,model_latencies_avg,model_latencies_p99,throughput_max,expected_max_throughput
0,yolov5m,1,1,20,0.293608,0.347349,3.370005,2.878951
